In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib, json
import matplotlib.pyplot as plt
import sys
sys.path.append('/Users/BrunoMattos/Documents2/Dev/stocklab_data/Firestore')
from uploadToFirestore import uploadDocumentToFirestore
import os
#print(os.listdir("./dre_input"))
itr_input = './input_cvm/itr/'
dfp_input = './input_cvm/dfp/'
# print(os.listdir("./input_cvm/itr"))

In [164]:
MapaNiveis = {
    'Caixa Líquido Atividades Operacionais': 1,
        'Caixa Gerado nas Operações': 1.1,
        'Variações nos Ativos e Passivos': 1.2,
        'Outros': 1.3,
    'Caixa Líquido Atividades de Investimento': 2,
    'Caixa Líquido Atividades de Financiamento': 3,
    'Variação Cambial s/ Caixa e Equivalentes':4 ,
    'Aumento (Redução) de Caixa e Equivalentes': 5,
        'Saldo Inicial de Caixa e Equivalentes': 5.1,
        'Saldo Final de Caixa e Equivalentes': 5.2, 
}
MapNivel1 = {
    'Caixa Líquido Atividades Operacionais':'Caixa Líquido Atividades Operacionais',
    'Caixa Líquido Atividades de Investimento':'Caixa Líquido Atividades de Investimento',
    'Caixa Líquido Atividades de Financiamento':'Caixa Líquido Atividades de Financiamento',
}
MapNivel2 = {
    'Caixa Gerado nas Operações':'Caixa Gerado nas Operações',
    'Variações nos Ativos e Passivos':'Variações nos Ativos e Passivos',
    'Outros':'Outros',
    'Saldo Inicial de Caixa e Equivalentes':'Saldo Inicial de Caixa e Equivalentes',
    'Saldo Final de Caixa e Equivalentes':'Saldo Final de Caixa e Equivalentes',   
}
MapGrupo = {
    'DF Consolidado - Demonstração do Fluxo de Caixa (Método Indireto)': 'Consolidado',
    'DF Individual - Balanço Patrimonial Ativo': 'Ativo Individual',
}

In [8]:
#DEMONSTRAÇÃO DE FLUXO DE CAIXA - DFC

# --- ITR
DFC_ITR_CON = pd.DataFrame() 
path = itr_input + 'itr_cia_aberta_dfc_mi_con_'
for year in range(2015,2020,1):
    DFC_ITR_CON = pd.concat([DFC_ITR_CON, pd.read_csv(path+str(year)+'.csv', thousands=',', sep=';', encoding='latin-1')])

DFC_ITR_IND = pd.DataFrame()
path = itr_input + 'itr_cia_aberta_dfc_mi_ind_'
for year in range(2015,2020,1):
    DFC_ITR_IND = pd.concat([DFC_ITR_IND, pd.read_csv(path+str(year)+'.csv', thousands=',', sep=';', encoding='latin-1')])

# --- DFP
# DFC_DFP_CON = pd.DataFrame() 
# path = dfp_input + 'bpa_cia_aberta_con_'
# for year in range(2015,2020,1):
#     DFC_DFP_CON = pd.concat([DFC_DFP_CON, pd.read_csv(path+str(year)+'.csv', thousands=',', sep=';', encoding='latin-1')])

# DFC_DFP_IND = pd.DataFrame()
# path = dfp_input + 'bpa_cia_aberta_ind_'
# for year in range(2015,2020,1):
#     DFC_DFP_IND = pd.concat([DFC_DFP_IND, pd.read_csv(path+str(year)+'.csv', thousands=',', sep=';', encoding='latin-1')])

# DFC = pd.concat([DFC_ITR_CON, DFC_ITR_IND, DFC_DFP_CON, DFC_DFP_IND]) 
DFC = pd.concat([DFC_ITR_CON, DFC_ITR_IND]) 

In [20]:
DFC.GRUPO_DFP.unique()

array(['DF Consolidado - Demonstração do Fluxo de Caixa (Método Indireto)',
       'DF Individual - Demonstração do Fluxo de Caixa (Método Indireto)'],
      dtype=object)

In [14]:
DFC.groupby(['CD_CONTA','DS_CONTA']).CD_CVM.count()

CD_CONTA  DS_CONTA                                                     
6.01      Caixa Líquido Atividades Operacionais                            27297
6.01.01   Caixa Aplicado nas Operações                                         2
          Caixa Gerado nas Operacoes                                           8
          Caixa Gerado nas Operações                                       27285
          Caixa proveniente (aplicado nas) das operações                       2
6.01.02   Variacoes nos Ativos e Passivos                                      8
          Variações nos Ativos e Obrigações                                    4
          Variações nos Ativos e Passivos                                  27261
          Variações nos Ativos e Passivos Operacionais:                        8
          Variações nos ativos e passivos                                      8
          Variações nos ativos e passivos, circulante e não circulantes        8
6.01.03   Outros                     

In [194]:
#Drop ORDEM_EXERC==PENÚLTIMO
BP=BP[BP.ORDEM_EXERC=='ÚLTIMO']
print('1 - Gerando NÍVEIS ...')
def getLevel(row):
    return len(row['CD_CONTA'].split('.'))   
BP['NIVEL']=BP.apply(lambda r: getLevel(r), axis=1)

1 - Gerando NÍVEIS ...


In [195]:

#Filtrar para somente NIVEL 1
print('2 - Filtrar para somente NIVEL 1')
BP_NIVEL1 = BP[BP.NIVEL==1]
BP_NIVEL1['DESC_SIMPLES']=BP_NIVEL1['DS_CONTA'].map(MapNivel1)
BP_NIVEL1.dropna(subset=['DESC_SIMPLES'], inplace=True)
BP_NIVEL1['CD_CONTA']=BP_NIVEL1['DESC_SIMPLES'].map(MapaNiveis)
#Filtrar para somente NIVEL 2
print('3 - Filtrar para somente NIVEL 2')
BP_NIVEL2 = BP[BP.NIVEL==2]
BP_NIVEL2['DESC_SIMPLES']=BP_NIVEL2['DS_CONTA'].map(MapNivel2)
BP_NIVEL2.dropna(subset=['DESC_SIMPLES'], inplace=True)
BP_NIVEL2['CD_CONTA']=BP_NIVEL2['DESC_SIMPLES'].map(MapaNiveis)
#Filtrar para somente NIVEL 3
print('4 - Filtrar para somente NIVEL 3')
BP_NIVEL3 = BP[BP.NIVEL==3]
#Alterando 'Empréstimos e Financiamentos do Passivo Não Circulante para diferenciar do Passivo Circulante
mask = ((BP_NIVEL3.DS_CONTA=='Empréstimos e Financiamentos') & (BP_NIVEL3.CD_CONTA=='2.02.01'))
BP_NIVEL3.loc[mask,'DS_CONTA'] = 'Empréstimos e Financiamentos a Longo Prazo'
BP_NIVEL3['DESC_SIMPLES']=BP_NIVEL3['DS_CONTA'].map(MapNivel3)
BP_NIVEL3.dropna(subset=['DESC_SIMPLES'], inplace=True)
BP_NIVEL3['CD_CONTA']=BP_NIVEL3['DESC_SIMPLES'].map(MapaNiveis)
#Concat Nivel 1,2 e 3
print('5 - Concatenando todos níveis')
BP=pd.concat([BP_NIVEL1,BP_NIVEL2,BP_NIVEL3])
#Map Grupo (Individual, Consolidado)
print('6 - Finalizando')
BP['GRUPO_DFP']=BP['GRUPO_DFP'].map(MapGrupo)
#Mapeamento da Escala dos Valores
BP['ESCALA'] = BP.ESCALA_MOEDA.map({'MIL': 1000, 'MILHAR': 1000, 'UNIDADE': 1})
#FIM
BP.DT_FIM_EXERC=pd.to_datetime(BP.DT_FIM_EXERC)
#Drop colunas desnecessárias
BP.drop(['DT_REFER', 'VERSAO', 'ESCALA_MOEDA', 'MOEDA', 'ORDEM_EXERC', 'DS_CONTA'], axis=1, inplace=True)
    

2 - Filtrar para somente NIVEL 1
3 - Filtrar para somente NIVEL 2
4 - Filtrar para somente NIVEL 3
5 - Concatenando todos níveis
6 - Finalizando


In [196]:
def getTrim(month):
    if(month==2 or month==3):
        return 1
    elif(month==5 or month==6):
        return 2
    elif(month==8 or month==9):
        return 3
    elif(month==11 or month==12):
        return 4
    else:
        return np.nan

BP['TRIM'] = BP.DT_FIM_EXERC.map(lambda d: getTrim(d.month))
BP.dropna(inplace=True)
BP['YEAR'] = BP.DT_FIM_EXERC.dt.year

In [198]:
def calculateAV(r):
    yr = r.YEAR
    trim = r.TRIM
    valor = r.VL_CONTA
    ativo_total = df[(df.YEAR==yr) & (df.TRIM == trim) & (df.CD_CONTA == '1')]
    passivo_total = df[(df.YEAR==yr) & (df.TRIM == trim) & (df.CD_CONTA == '2')]
    if(len(ativo_total)):
        baseValue = ativo_total['VL_CONTA'].values[0]
    elif(len(passivo_total)):
        baseValue = passivo_total['VL_CONTA'].values[0]
    else:
        baseValue=0
    if baseValue!=0:
        return valor/baseValue
    else:
        return np.nan


In [203]:
numberOfCVM = BP.CD_CVM.nunique()
count=0
for cvm in BP.CD_CVM.unique()[70:numberOfCVM+1]:
    # cvm=4170
    count=count+1
    print(str(count) + '/' + str(numberOfCVM) + ' - cvm: ' + str(cvm))
    DF = BP[(BP.CD_CVM==cvm)]
    for grupo in ['Consolidado', 'Individual']:
        print('Grupo: ' + grupo)
        GrupoAP = pd.DataFrame([])
        for subgrupo in ['Ativo Consolidado', 'Passivo Consolidado','Ativo Individual', 'Passivo Individual']:
            if(grupo in subgrupo):
                # print(' - Sub grupo: ' + subgrupo)
                df = DF[DF.GRUPO_DFP == subgrupo]
                if(len(df)):
                    cnpj =  df.iloc[0]['CNPJ_CIA']  
                    denom_cia =  df.iloc[0]['DENOM_CIA']
                    df = df.sort_values(['DESC_SIMPLES', 'DT_FIM_EXERC'])
                    df.reset_index(inplace=True, drop=True)
                    
                    #Drop columns
                    df.drop(['DT_FIM_EXERC', 'CNPJ_CIA'], axis=1, inplace=True)
                    
                    #Sort para garantir o sequenciamento correto
                    df = df.sort_values(['DESC_SIMPLES', 'YEAR','TRIM']).reset_index(drop=True)
                    #Calculate Variação entre Trimestre - AH (Análise Horizontal)
                    cutTwo = (df.DESC_SIMPLES == df.DESC_SIMPLES.shift(1)) 
                    AH = []
                    for i, row in df.iterrows():
                        if cutTwo[i]:
                            if df.iloc[i-1]['VL_CONTA']!=0:
                                ah = (df.iloc[i]['VL_CONTA']-df.iloc[i-1]['VL_CONTA'])/df.iloc[i-1]['VL_CONTA']
                            else:
                                ah = np.nan
                            AH.append(ah)
                        else:
                            AH.append(np.nan)
                    df['AH']=pd.Series(AH)
                    #Sort novamente
                    df = df.sort_values(['YEAR','TRIM','CD_CONTA']).reset_index(drop=True)
                    #Cálculo da Variação Vertical no Trimestre - AV (Análise Vertical)
                    df['AV'] = df.apply(lambda r: calculateAV(r), axis=1)
                    #Adicionando o TRIM_ANO como coluna
                    df['TRIM_ANO']=df.apply(lambda r: "%dT%d" %(r['TRIM'],r['YEAR']), axis=1)
                    #Concat Ativo e Passivo de mesmo grupo
                    GrupoAP = pd.concat([GrupoAP,df])
                else:
                    print('  ' + str(cvm) +  ' - ' + grupo +' - ' + 'EMPTY')
    
        if(len(GrupoAP)>0):
            for ano in GrupoAP.YEAR.unique():
                filteredDFbyYear = GrupoAP[GrupoAP.YEAR==ano]
                #Gravando arquivo
                if(len(filteredDFbyYear)):
                    filteredDFbyYear = filteredDFbyYear[['TRIM_ANO','CD_CONTA','DESC_SIMPLES', 'VL_CONTA', 'ESCALA', 'TRIM', 'YEAR', 'NIVEL', 'AH', 'AV']]
                    filteredDFbyYear=filteredDFbyYear.sort_values(['TRIM_ANO','CD_CONTA']).reset_index(drop=True)
                    # print(filteredDFbyYear[['TRIM_ANO','CD_CONTA','DESC_SIMPLES', 'VL_CONTA', 'AH', 'AV']].head(50))
                    data = filteredDFbyYear.to_dict('records')
                else:
                    data = []           
                data = {
                    "cvm": int(cvm),
                    "ano": int(ano),
                    "cnpj": cnpj,
                    "denom_cia": denom_cia,
                    "grupo": grupo,
                    "data" : data,
                }
                path = './output_cvm/bp/'+str(cvm)+'/'+grupo+'/'
                if not os.path.exists(path):
                    os.makedirs(path)
                with open(path+str(ano)+'.json', 'w') as outfile:
                    json.dump(data, outfile, indent=2)
                    print('  '+ str(cvm) +  ' - ' + grupo +' - ' + str(ano)+'.json gravado - ' + str(df[df.YEAR==ano].size))


5
Grupo: Consolidado
  8605 - Consolidado - EMPTY
  8605 - Consolidado - EMPTY
Grupo: Individual
  8605 - Individual - 2015.json gravado - 520
  8605 - Individual - 2016.json gravado - 520
  8605 - Individual - 2017.json gravado - 520
  8605 - Individual - 2018.json gravado - 520
  8605 - Individual - 2019.json gravado - 390
610/753 - cvm: 15113
Grupo: Consolidado
  15113 - Consolidado - EMPTY
  15113 - Consolidado - EMPTY
Grupo: Individual
  15113 - Individual - 2015.json gravado - 520
  15113 - Individual - 2016.json gravado - 390
611/753 - cvm: 16861
Grupo: Consolidado
  16861 - Consolidado - EMPTY
  16861 - Consolidado - EMPTY
Grupo: Individual
  16861 - Individual - 2015.json gravado - 520
  16861 - Individual - 2016.json gravado - 520
  16861 - Individual - 2017.json gravado - 520
  16861 - Individual - 2018.json gravado - 520
  16861 - Individual - 2019.json gravado - 390
612/753 - cvm: 8117
Grupo: Consolidado
  8117 - Consolidado - EMPTY
  8117 - Consolidado - EMPTY
Grupo: Indi